In [1]:
from shared.preface import *
import shared.functions as fct

## Ch. 0: Preliminary Investigations.

### 2D grid, center coord. pairs.

In [ ]:
def grid_2D(l, s):

    # 2D spatial grid, discretized.
    eps = s/10
    x, y = np.mgrid[-l:l+eps:s, -l:l+eps:s]

    # [x y] edge coordinate pairs of above grid.
    xy = np.mgrid[-l:l+eps:s, -l:l+eps:s].reshape(2,-1).T

    # Create center coord.-pairs.
    x_centers = (x[1:,:] + x[:-1,:])/2.
    y_centers = (y[:,1:] + y[:,:-1])/2.
    centers = np.array([x_centers[:,:-1], y_centers[:-1,:]])
    cent_coordPairs2D = centers.reshape(2,-1).T
    print('All coord. pairs 2D:\n', cent_coordPairs2D)
    # print('Coord pairs 2D shape:', cent_coordPairs2D.shape)

    return cent_coordPairs2D

limit_coarse, space_coarse = 1.5, 1.
cent_coordPairs2D = grid_2D(limit_coarse, space_coarse)

# Delete middle square.
cent_coordPairs2D = np.delete(cent_coordPairs2D, 4, axis=0)

# Create finegrained square.
limit_fine, space_fine = limit_coarse/2., space_coarse/2.
cent_coordPairs2D_fine = grid_2D(limit_fine, space_fine)

# Insert finegrained square.

### 3D grid, center coord. pairs.

In [ ]:
# 3D spatial grid, discretized.
x, y, z = np.mgrid[-1:1.1:1., -1:1.1:1., -1:1.1:1.]
# print(x[0,...], x.shape)

x_centers = (x[1:,...] + x[:-1,...])/2.
# print(x_centers, x_centers.shape)

y_centers = (y[:,1:,:] + y[:,:-1,:])/2.
# print(y_centers, y_centers.shape)

z_centers = (z[...,1:] + z[...,:-1])/2.
# print(z_centers, z_centers.shape)


# Create center coord.-pairs., truncate redundant points.
centers3D = np.array([
    x_centers[:,:-1,:-1], 
    y_centers[:-1,:,:-1], 
    z_centers[:-1,:-1,:]
])
# print(centers3D, centers3D.shape)

cent_coordPairs3D = centers3D.reshape(3,-1).T 
print(cent_coordPairs3D, cent_coordPairs3D.shape)

## Ch. 1: Milky Way-type halo and simple grid.

### 3D plot of the DM particles.

In [ ]:
# Read DM particles.
fct.read_DM_positions_randomHalo(which_halos='halos', mass_select=12)

# Build grid around Milky Way.
MW_grid = fct.grid_3D(GRID_L, GRID_S) / kpc

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

# Read in DM particle positions.
DM_pos = np.load('sim_data/DM_positions_halos_M12.npy')
print(f'{len(DM_pos)}')
x_DM, y_DM, z_DM = DM_pos[:,0], DM_pos[:,1], DM_pos[:,2]
cut = 10
x, y, z = x_DM[1::cut], y_DM[1::cut], z_DM[1::cut]

ax.scatter(x, y, z, alpha=0.1, c='dodgerblue')

# Draw sphere around GC with radius=Rvir_MW.
rGC = Rvir_MW/kpc
uGC, vGC = np.mgrid[0:2 * np.pi:200j, 0:np.pi:100j]
xGC = rGC * np.cos(uGC) * np.sin(vGC)
yGC = rGC * np.sin(uGC) * np.sin(vGC)
zGC = rGC * np.cos(vGC)

xg, yg, zg = MW_grid[:,0], MW_grid[:,1], MW_grid[:,2] 
ax.scatter(xg, yg, zg, s=0.2, marker='x', color='black', alpha=0.5)


ax.plot_surface(
    xGC, yGC, zGC, alpha=0.1, 
    cmap=plt.cm.coolwarm, vmin=-1, vmax=1,# antialiased=False,
    rstride=1, cstride=1
)

plt.show()

### Calculate gravity in each cell.

In [ ]:
DM_pos = np.load('sim_data/DM_positions_halos_M12.npy')

### Testing 1 cell with coords. at earth.
cell_id = 0
cell1 = np.array([8.5, 0, 0])*kpc

cell_vector = fct.cell_gravity(cell1, DM_pos, GRAV_RANGE, DM_SIM_MASS)
cell_vector /= (kpc/s**2)  
print(cell_vector)
print(np.sqrt(np.sum(cell_vector**2)))

#! this should be around same order of magnitude as in spher. symmetric setup.

## Ch. 2: Broadcasting for cell_gravity.

In [ ]:
fct.read_DM_positions_randomHalo(which_halos='halos', mass_select=12)
DM = np.load('sim_data/DM_positions_halos_M12.npy')[1::10]
grid = fct.grid_3D(GRID_L, GRID_S)

grid = np.expand_dims(grid, axis=1)

DM = np.expand_dims(DM, axis=0)
DM = np.repeat(DM, len(grid), axis=0)

# diff = DM - grid
# print(diff.shape, (diff.nbytes)/1e6)

fct.cell_gravity_3D(grid, DM, GRAV_RANGE, DM_SIM_MASS)
dPsi_grid = np.load('CubeSpace/dPsi_grid_snapsnot_X.npy')
dPsi_grid /= (kpc/s**2) 
mags = np.sqrt(np.sum(dPsi_grid**2, axis=1))
print(mags[0])

## Ch. 3: Combined precalculations for all snapshots.

In [ ]:
from shared.preface import *
import shared.functions as fct

for i in range(12,37):
    snap_i = f'{i:04d}'
    print(snap_i)

    # 1. Read in DM positions of halo in snapshot.
    fct.read_DM_positions_randomHalo(
        which_halos='halos', mass_select=12, mass_range=1., snap_num=snap_i
        )
    DM_pos = np.load('sim_data/DM_positions_halos_M12.npy')[1::10]

    # 2. Build the spatial grid, depending on virial radius of halo, etc.
    cell_grid = fct.grid_3D(GRID_L, GRID_S)
    np.save(f'CubeSpace/cell_grid_snapsnot_{snap_i}', cell_grid)
    cell_grid = np.expand_dims(cell_grid, axis=1)

    # 2.5 Adjust arrays.
    DM_pos = np.expand_dims(DM_pos, axis=0)
    DM_pos = np.repeat(DM_pos, len(cell_grid), axis=0)

    # 3. Calculate derivatives of each cell.
    fct.cell_gravity_3D(
        cell_grid, DM_pos, GRAV_RANGE, DM_SIM_MASS, snap_num=snap_i
        )

snapshot_0012 is at redshift ~3.8.

**BUT** it contains no halo of mass_select=12 anymore.

**TODO:** start at snapshot_0036 (z~1e-16) and **trace same halo** backwards through the
snapshots.

Ugly fix for now: Extended mass_select window to search down to 1e11*Msun halos.
Then just pick random halo, will not be the same in all snapshots.

### Redshift z of each snapshot.

In [ ]:
zeds = np.zeros(25)
nums = []
for j, i in enumerate(range(12,37)):
    snap_i = f'{i:04d}'
    nums.append(snap_i)

    with h5py.File(f'{SIM_DATA}/snapshot_{snap_i}.hdf5') as snap:
        zeds[j] = snap['Cosmology'].attrs['Redshift'][0]

np.save(f'shared/ZEDS_SNAPSHOTS.npy', np.asarray(zeds))
np.save(f'shared/NUMS_SNAPSHOTS.npy', np.asarray(nums))

## Ch. 4: Better derivative grid structure. 

### Final function: compile everything here later...

In [ ]:
# Function for grouping DM particles around cells.
# Framed differently: how much DM particles are around each cell?

def DMnum_around_cell(cell_length):
    """Amount of DM particles in sphere around cell."""

    # Center all DM positions w.r.t. cell center.
    DM_cc = DM_coords*kpc - cell_coords

    # Calculate distances of DM to cc.
    DM_dist = np.sqrt(np.sum(DM_cc**2, axis=2))

    # Ascending order indices.
    ind = DM_dist.argsort(axis=1)
    ind_3D = np.expand_dims(ind, axis=2)
    ind_3D = np.repeat(ind_3D, 3, axis=2)

    # Sort DM positions according to dist.
    DM_pos_sort = np.take_along_axis(DM_cc, ind_3D, axis=1)
    DM_dist_sort = np.take_along_axis(DM_dist, ind, axis=1)

    # Keep DM inside certain range.
    DM_pos_inRange = DM_pos_sort[DM_dist_sort <= grav_range]


    # recursive structure most likely...
    # DMnum_around_cell(new_cell_length)
    # ...


### steps until: DM_pos array (cells, DM_amount, 3) containing nans.

In [2]:
cell_num, DM_num = 3, 6
DM_cc = np.random.random(size=(cell_num, DM_num, 3)) * 5
# print('DM_cc', DM_cc.shape)

print(DM_cc)

# Calculate distances of DM to cc.
DM_dist = np.sqrt(np.sum(DM_cc**2, axis=2))

# Ascending order indices.
ind = DM_dist.argsort(axis=1)
ind_3D = np.expand_dims(ind, axis=2)
ind_3D = np.repeat(ind_3D, 3, axis=2)

# Sort DM positions according to dist.
DM_pos_sort = np.take_along_axis(DM_cc, ind_3D, axis=1)
DM_dist_sort = np.take_along_axis(DM_dist, ind, axis=1)
# print('DM_pos_sort', DM_pos_sort.shape)
# print('DM_dist_sort', DM_dist_sort.shape)

# Keep DM inside certain range.
radius = 5

# Index array for DM inside radius, for each cell: 
# [[cell_num, DM_particle_in_cell],...] (see print)
trunc_ind = np.argwhere(DM_dist_sort <= radius)
# print('trunc_ind \n', trunc_ind)

# Find index, up to which DM particles should be kept for each cell.
row = trunc_ind.T[1]
fw_diff = row[:-1] - row[1:]
# print(fw_diff)
take = np.argwhere(fw_diff.flatten() >= 0).flatten()
# print(take)

# -> Index up to max DM particle inside radius, paired with each cell num: 
# [[cell_0, DM_max_rank_for_cell_1],...] (see print)
selection = np.vstack((trunc_ind[take,:], trunc_ind[-1,:]))
print(selection, selection.shape)

# Replace all entries beyond this indices (for each cell) with nan values.
for i in range(cell_num):
    DM_cc[i, selection[i,1]+1:, :] = np.nan

print(DM_cc)

# DM_pos_inRange = DM_pos_sort[selection[:,0], :selection[:,1], :]
# print('DM_pos_inRange', DM_pos_inRange.shape)

[[[3.40174463 0.83022935 3.50646878]
  [3.81254045 1.07734463 3.69189687]
  [1.48754678 0.79140075 2.66301873]
  [2.85669883 0.5781353  0.37083844]
  [1.71107986 1.47976177 1.03260905]
  [0.37428074 4.52022854 3.52353901]]

 [[2.01268252 1.51130776 4.09996638]
  [3.65392053 1.80047704 4.82493029]
  [2.10836856 3.47406539 0.33554428]
  [3.65153456 4.16868702 1.11156161]
  [2.023621   1.70308657 0.75720067]
  [1.16030079 1.16624842 1.84911662]]

 [[1.74777379 3.87590252 3.41629203]
  [0.50272365 3.05094146 1.16615695]
  [4.53132351 2.74682568 1.531754  ]
  [2.95599237 0.69180604 1.99857213]
  [0.77181338 4.36940132 1.93926686]
  [3.39828885 2.53797609 4.41778309]]]
[[0 3]
 [1 3]
 [2 2]] (3, 2)
[[[3.40174463 0.83022935 3.50646878]
  [3.81254045 1.07734463 3.69189687]
  [1.48754678 0.79140075 2.66301873]
  [2.85669883 0.5781353  0.37083844]
  [       nan        nan        nan]
  [       nan        nan        nan]]

 [[2.01268252 1.51130776 4.09996638]
  [3.65392053 1.80047704 4.82493029]
 

### steps until: deleting common nan rows, and cells not containing enough DM, and resetting DM positions.

In [3]:
# Drop all "rows", shared by all cells, which have only nan values,
# determined by maximum present value in DM rank of all cells.

# 1. Find max index (for DM rank) present in selection array.
max_DM_rank = np.max(selection[:,1])
print(max_DM_rank)

# 2. "Delete" those common rows. 
#! np. delete creates a new array, so delete old and free memory (just in case). 
DM_cc_compact = np.delete(DM_cc, np.s_[max_DM_rank+1:], axis=1)

# with contextlib.redirect_stdout(None):
#     del DM_cc
#     gc.collect()

print(DM_cc_compact)

# Counting how many (non-)nan rows are in each cell.
DM_particle_count = np.count_nonzero(~np.isnan(DM_cc_compact[:,:,0]), axis=1)
nan_count = np.count_nonzero(np.isnan(DM_cc_compact[:,:,0]), axis=1)
print(DM_particle_count, nan_count)


# Drop all cells, which have DM particle amount lower than threshold.
#note: see later which is more efficient, drop cells from DM_compact or before.
bool_DM = DM_particle_count < 2
print(bool_DM)

DM_cc_cell_filtered = np.delete(DM_cc_compact, bool_DM, axis=0)
print(DM_cc_cell_filtered)

# Also filter the cell coordinates. 
proxy_cell_coords = np.arange(cell_num*3).reshape((cell_num, 1, 3))
cell_coords_filtered = np.delete(proxy_cell_coords, bool_DM, axis=0)

print(DM_cc_cell_filtered.shape, cell_coords_filtered.shape)

# Reset the DM coords. 
DM_cc_reset = DM_cc_cell_filtered + cell_coords_filtered

3
[[[3.40174463 0.83022935 3.50646878]
  [3.81254045 1.07734463 3.69189687]
  [1.48754678 0.79140075 2.66301873]
  [2.85669883 0.5781353  0.37083844]]

 [[2.01268252 1.51130776 4.09996638]
  [3.65392053 1.80047704 4.82493029]
  [2.10836856 3.47406539 0.33554428]
  [3.65153456 4.16868702 1.11156161]]

 [[1.74777379 3.87590252 3.41629203]
  [0.50272365 3.05094146 1.16615695]
  [4.53132351 2.74682568 1.531754  ]
  [       nan        nan        nan]]]
[4 4 3] [0 0 1]
[False False False]
[[[3.40174463 0.83022935 3.50646878]
  [3.81254045 1.07734463 3.69189687]
  [1.48754678 0.79140075 2.66301873]
  [2.85669883 0.5781353  0.37083844]]

 [[2.01268252 1.51130776 4.09996638]
  [3.65392053 1.80047704 4.82493029]
  [2.10836856 3.47406539 0.33554428]
  [3.65153456 4.16868702 1.11156161]]

 [[1.74777379 3.87590252 3.41629203]
  [0.50272365 3.05094146 1.16615695]
  [4.53132351 2.74682568 1.531754  ]
  [       nan        nan        nan]]]
(3, 4, 3) (3, 1, 3)


### steps for: creating coord array for sub 8 division of cells.

In [4]:
orig_GRID_S = 5

test_grid = fct.grid_3D(orig_GRID_S/2., orig_GRID_S/2)
print(test_grid)

#note CONTINUE HERE: 1. the subdivision has to be done for many cells, each
#note                   having different coords -> find broadcast-style method?
#note                2. continue with steps (see goodnotes), loop almost done!

[[-1.25 -1.25 -1.25]
 [-1.25 -1.25  1.25]
 [-1.25  1.25 -1.25]
 [-1.25  1.25  1.25]
 [ 1.25 -1.25 -1.25]
 [ 1.25 -1.25  1.25]
 [ 1.25  1.25 -1.25]
 [ 1.25  1.25  1.25]]
